In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from src.utils.images import plot_matrix, debug, plot_array_matrix
%autosave 60

In [ ]:
from src.utils.datasets import CKDataset
size = 96
# ck_dataset = CKDataset()
gaussians_inputs = foo(3000)

In [ ]:
from src.supervised_gcal.models import get_full_lissom
from src.supervised_gcal.optimizers import SequentialOptimizer, SimpleHebbian, NeighborsDecay
input_shape = (size, size)
cortex_shape = input_shape
lgn_shape = cortex_shape

radius_aff = 8
radius_excit = 5
radius_inhib = 10
pruning_step = len(gaussians_inputs)-1
v1_params = {'min_theta':0.1, 'max_theta':0.65, 'afferent_radius':radius_aff,
                                             'excitatory_radius':radius_excit, 'inhibitory_radius':radius_inhib, 
                                             'settling_steps':9, 'inhib_factor':0.9, 'excit_factor':0.9}

lissom, _, _ = get_full_lissom(input_shape, lgn_shape, cortex_shape, v1_params=v1_params)
optimizer = SequentialOptimizer(
                SimpleHebbian(cortex_layer=lissom.v1, learning_rate=0.9590, weight='afferent_weights'),
                SimpleHebbian(cortex_layer=lissom.v1, learning_rate=2.55528, weight='excitatory_weights'),
                SimpleHebbian(cortex_layer=lissom.v1, learning_rate=1.80873, weight='inhibitory_weights'))

In [ ]:
from src.utils.pipeline import Pipeline
from torch.utils.data import DataLoader
train_loader = DataLoader(gaussians_inputs, shuffle=True, pin_memory=True)
pipe = Pipeline(lissom, optimizer, cuda=True, log_interval=100)

In [ ]:
from src.utils.datasets import CKDataset
size = 96
ck_dataset = CKDataset()
from torch.utils.data import DataLoader
train_loader = DataLoader(ck_dataset, shuffle=True, pin_memory=True)

In [ ]:
from src.utils.pipeline import Pipeline
lissom.cuda()
for inp, _ in train_loader:
    inp = torch.autograd.Variable(inp)
    inp = Pipeline.process_input(inp)
    inp = inp.cuda()
    act = lissom.on(inp)
    plot_matrix(np.reshape(act.data.cpu().numpy(), cortex_shape))
    act_old = act
    act = lissom.off(inp)
    act = act + act_old
    act = lissom(inp)
    break

In [ ]:
from tqdm import tqdm_notebook as tqdm
for epoch in tqdm(range(epochs)):
    pipe.train(train_loader, epoch)

In [ ]:
from src.supervised_gcal.utils.images import images_matrix
plot_matrix(np.transpose(images_matrix(lissom.v1.inhibitory_weights.data, (0,1)).cpu().numpy()*500, (1,2,0)))

In [ ]:
from src.supervised_gcal.utils.orientation_maps import calculate_keys_activations
keys_arrays = generate_all_inputs(size, orientations=13)
lissom.cuda()
activations = calculate_keys_activations(lissom, keys_arrays)